In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import re
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

df_test = pd.read_csv("../ventas_limpias.csv")

print(df_test)
# [1.048.574  rows x 11 columns]
# 11 Columnas
# 1.048.575 rows

In [7]:
#Carga el archivo .env
load_dotenv(override=True)

#Variables de entorno

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")


URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(URL)

conn = engine.connect()
try:
    print("Connection Succesfull!!" if conn else "")

except Exception as e:
    print("Error al conectar la base datos en", e)


Connection Succesfull!!


In [ ]:
#set = conn.execute(text("SET search_path TO riwi_ventas"))

df = pd.read_sql(text("SELECT * FROM factura_ventas;"),conn)
print(df)




         venta_id       fecha  ciudad_id  producto_id  tipo_venta_id  \
0               1  2025-10-30          1            1              1   
1               2  2025-11-17          2            1              2   
2               3  2025-10-22          3            2              2   
3               4  2025-10-20          4            3              3   
4               5  2025-10-20          5            4              2   
...           ...         ...        ...          ...            ...   
1048570   1048571  2025-10-23         18           53              1   
1048571   1048572  2025-11-13         14           39              3   
1048572   1048573  2025-10-30         11           71              5   
1048573   1048574  2025-10-23          5           41              1   
1048574   1048575  2025-10-22         24            8              4   

         tipo_cliente_id  cantidad  precio_unitario  descuento  costo_envio  \
0                      1       2.0           3681.0     